<a href="https://colab.research.google.com/github/JonaFlavier/Learning/blob/main/Python/TextBased/ChatBot/dynamic_chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from openai import OpenAI
import tiktoken
import json
from datetime import datetime
from env_variables import OPENAI_API_KEY

In [ ]:
# api_key = os.environ["OPENAI_API_KEY"]
api_key = OPENAI_API_KEY
BASE_URL = "https://api.openai.com/v1"
model_name ="gpt-3.5-turbo"
model_name_1 = "gpt-3.5-turbo-0125"


In [ ]:
# advised to handle API Keys securely (ie as env variables in a .env file)
class ConversationManager:
    # handle interactions with the AI model
    def __init__(self, api_key=OPENAI_API_KEY,base_url=BASE_URL, history_file=None, default_model=model_name):
        # initialise the chat bot
        self.openai_client = OpenAI(api_key= OPENAI_API_KEY, base_url= BASE_URL)
        self.api_key = api_key
        self.base_url = base_url
        self.default_model = default_model
        self.system_msg_dict = {
            "sassy_persona":"You are a sassy assistant who is fed up with answering questions.",
            "coding_persona": "You are a coding assistant who can help with improving code structures, software architecture pipelines and making sure that everything is industry standard",
            "research_persona": "You are a research assistant that helps to simplify and summarise research topics and concepts and suggests ideas"
            }
        self.system_msg = self.system_msg_dict["research_persona"]

        if history_file is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            self.history_file = f"conversation_history_{timestamp}.json"
        else:
            self.history_file = history_file
        # load the conversation from the file or create a file if the file doesnt exist
        self.load_conversation_history()


    def set_persona(self, persona):
        if persona in self.system_msg_dict:
            self.system_msg = self.system_msg_dict[persona]
            self.update_system_message_in_history()
        else:
            raise ValueError(f"Unknown persona: {persona}. Available personas are: {list(self.system_msg_dict.keys())}")

    def set_custom_system_msg(self, custom_msg):
        if not custom_msg:
            raise ValueError("Custom message cannot be empty")
        self.system_msg_dict['custom'] = custom_msg
        self.set_persona('custom')
    def update_system_message_in_history(self):
        try:
            if self.conversation_history and self.conversation_history[0]["role"] == "system":
                self.conversation_history[0]["content"] = self.system_msg
            else:
                self.conversation_history.insert(0, {"role":"system", "content": self.system_msg})
        except Exception as e:
            print(f"Unexpected error while updating the system message in the conversation history: {e}")

    def chat_completion(self, user_prompt):
        # send user's prompt to AI and wait for the response
        ai_response = user_prompt
        try:
            # get a response from API
            response = self.openai_client.chat.completions.create(
                model=model_name_1,
                messages=[{"role":"user", "content": user_prompt}]
            )
            print(response)
        except Exception as e:
            print(f"error during response generation: {e}")
            return None

        ai_response = response.choices[0].message.content
        self.conversation_history.append({"role": "assistant", "content": ai_response})
        self.save_conversation_history()
        return ai_response

    # loads conversation history from file
    def load_conversation_history(self):
        try:
            # with every line in file read contents
            with open(self.history_file, 'r') as file:
                self.conversation_history = json.load(file)
        except FileNotFoundError:
            self.conversation_history = [{"role": "system", "content": self.system_msg}]
        except json.JSONDecodeError:
            print("Error reading the conversation history file. Starting with an empty history.")
            self.conversation_history = [{"role": "system", "content": self.system_msg}]


    # saves conversation history to the file
    def save_conversation_history(self):
        try:
            with open(self.history_file, "w") as file:
                json.dump(self.conversation_history, file, indent=4)
        except IOError as e:
            print(f"IO error while saving conversation history: {e}")
        except Exception as e:
            print(f"Unexpected error occured while saving the conversation history: {e}")

    # reset the conversation history
    def reset_conversation_history(self):
        self.save_conversation_history = [{"role":"system", "content": self.system_message}]
        try:
            self.save_conversation_history()
        except Exception as e:
            print(f"Unexpected error occurred while resetting the conversation history: {e}")




In [ ]:
# initialise the chatbot
conv_manager = ConversationManager()

In [ ]:
#test the chat bot
conv_manager.chat_completion("Can you give me some information on ligands and precursors for nanomaterial synthesis in chemistry?")

error during response generation: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
